In [25]:
%cd ..
%load_ext autoreload
%autoreload 2

BASE_PATH = "/Users/arattinger/Projects/rts/rts/testdata/"
DATA_PATH = BASE_PATH + "data/"
bucket_name = "rts"

/Users/arattinger/Projects/rts
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
from datetime import datetime
import orjson
import pandas as pd
import zipfile
import os
import io
from typing import Dict, List, Optional, Tuple
from pathlib import Path
from storage3.utils import StorageException

In [3]:
from supabase import create_client, Client

In [4]:
df = pd.read_hdf(os.path.join(BASE_PATH, 'rts_aivectors.hdf5'))

In [5]:
df.iloc[0]

guid                                                     AA1104002390
mediaFolderPath                               /mnt/rts/5/1/0/ZE004015
mediaDuration                                                    4437
ratio                                                            16:9
formatResolution                                                   SD
publishedDate                                    2011-04-09T00:00:00Z
categoryName                                                Programme
assetType                                                   Programme
contentType                                        Pop et rock, clips
backgoundType                                                    None
collection                                                  MusicOmax
publishedBy                                                     TSR 2
rights                                          Restriction/Condition
title               WALDER / PAMPLEMOUSSE / NICOLAS FRAISSINET - 1...
resume              

In [10]:
os.getenv("SUPABASE_HOST")

'localhost:8000'

In [8]:
os.getenv("SUPABASE_KEY")

'eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyAgCiAgICAicm9sZSI6ICJhbm9uIiwKICAgICJpc3MiOiAic3VwYWJhc2UtZGVtbyIsCiAgICAiaWF0IjogMTY0MTc2OTIwMCwKICAgICJleHAiOiAxNzk5NTM1NjAwCn0.dc_X5iR_VP_qT0zsiyj_I_OZ2T9FtRU2BBNWN8Bu4GE'

In [13]:
supabase: Client = create_client(
    "http://localhost:8000", 
    os.getenv("SUPABASE_KEY")
)

In [50]:
def upload_media(clip_df: pd.DataFrame):
    # Steps:
    # 1. Upload video clips
    # 2. Upload images

    # 1. Upload video clips
    # 1.1. Get list of video clips
    for index, row in clip_df.iterrows():
        video_path = row['mediaFolderPath'].split("/")[-1]
        video_path = os.path.join(DATA_PATH, video_path, 'clips', 'videos')
        # List all files in the directory
        files = os.listdir(video_path)

        print(video_path)
        for file in files:
            if file.endswith(".mp4"):
                print(file)
                file_path = os.path.join(video_path, file)
                # print(file_path)
                try:
                    supabase.storage.from_(bucket_name).upload(f"{bucket_name}/videos/{file}", file_path)
                except StorageException as e:
                    print(e.args[0]['error'])
                    if e.args[0]['error'] != 'Duplicate':
                        raise e
                    


        return
    




In [51]:
e = upload_media(df)

/Users/arattinger/Projects/rts/rts/testdata/data/ZE004015/clips/videos
ZE004015-L005.mp4
/Users/arattinger/Projects/rts/rts/testdata/data/ZE004015/clips/videos/ZE004015-L005.mp4
Duplicate
ZE004015-L004.mp4
/Users/arattinger/Projects/rts/rts/testdata/data/ZE004015/clips/videos/ZE004015-L004.mp4
Duplicate
ZE004015-L006.mp4
/Users/arattinger/Projects/rts/rts/testdata/data/ZE004015/clips/videos/ZE004015-L006.mp4
Duplicate
ZE004015-L003.mp4
/Users/arattinger/Projects/rts/rts/testdata/data/ZE004015/clips/videos/ZE004015-L003.mp4
Duplicate
ZE004015-L002.mp4
/Users/arattinger/Projects/rts/rts/testdata/data/ZE004015/clips/videos/ZE004015-L002.mp4
Duplicate
ZE004015-L000.mp4
/Users/arattinger/Projects/rts/rts/testdata/data/ZE004015/clips/videos/ZE004015-L000.mp4
Duplicate
ZE004015-L001.mp4
/Users/arattinger/Projects/rts/rts/testdata/data/ZE004015/clips/videos/ZE004015-L001.mp4
Duplicate


In [47]:
e.args[0]['error']

'Duplicate'